## Grupo 4 - Laboratorio 2 - Python

Katiuska Olivera Quevedo (20172533) <br/> 
Rosemery Fernandez Sanchez (20172635) <br/> 
Aliro Cabrera Florez (20152034) <br/> 
Jose Uscamayta Quispe (20195674) <br/> 

In [1]:
import pandas as pd
import numpy as np
import pyreadr
import math
import random
import os
import csv

### 1. Sample splitting to evaluate the performance of prediction rules to a fellow student and show how to use it on the wage data. 

Sample siplitting or data sampling it is a statistical analysis technique. This consist in the division of the sample (randomly) into two subsamples: train and test. 
To use it on the wage data, we are going to use the train sub sample and make a simple regression of $Y1$(lwage) and $X1$(sex). Of this regression we find a $\beta$. We are going to use this $\beta$ to calculate a $Y$ estimated. 
Using the real $Y2$ of the test sub sample and the $Y$ estimated we just found, now we are capable of finding an accurate MSE and R-squared of the regression.


### 2. Replicate the OLS and lasso for wage prediction- Jannis Kueck

Goals of the paper:

* Predict wages  using various characteristics of workers, in this case we are only using the data of the workers who didnt go to college (shs,hsg).

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.


We employ two different specifications for prediction:

1. Basic Model:  X  consists of a set of raw regressors 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'.

2. Flexible Model:  X  consists of all raw regressors from the basic model plus a dictionary of transformations ( lwage ~ (exp1 + exp2 + exp3 + exp4+ shs+ hsg+ scl+ clg+ occ2+ ind2+ mw+ so+ we)^2) and additional two-way interactions of a polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is experience times the indicator of not having a college degree.

Using the Flexible Model enables us to approximate the real relationship by a more complex regression model and therefore to reduce the bias. The Flexible Model increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver higher prediction accuracy but are harder to interpret.


#### 2.1 Data Analysis

In [2]:
rdata_read = pyreadr.read_r("C:/Users/ok235/OneDrive/Documentos/GitHub/ECO224/Labs/data/wage2015_subsample_inference.Rdata")
data = rdata_read[ 'data']
type(data)
data.shape

(5150, 20)

In [3]:
data.describe()

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4
count,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000
mean,23.410410,2.970787,0.444466,0.023301,0.243883,0.278058,0.317670,0.137087,0.259612,0.296505,0.216117,0.227767,13.760583,3.018925,8.235867,25.118038
std,21.003016,0.570385,0.496955,0.150872,0.429465,0.448086,0.465616,0.343973,0.438464,0.456761,0.411635,0.419432,10.609465,4.000904,14.488962,53.530225
min,3.021978,1.105912,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.461538,2.599837,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.250000,0.125000,0.062500
50%,19.230769,2.956512,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000,1.000000,1.000000
75%,27.777778,3.324236,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,21.000000,4.410000,9.261000,19.448100
max,528.845673,6.270697,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,47.000000,22.090000,103.823000,487.968100


In [4]:
#Sub group of the data 
Z = data[(data['shs'] == 1) | (data['hsg'] == 1) ]

In [5]:
Z.shape

(1376, 20)

In [6]:
Y = np.log2(Z['wage']) 
n = len(Y)
z = Z.loc[:, ~Z.columns.isin(['wage', 'lwage','Unnamed: 0'])]
p = z.shape[1]
print("Number of observation:", n, '\n')
print( "Number of raw regressors:", p)

Number of observation: 1376 

Number of raw regressors: 18


In [7]:
Z_1 = data.loc[:, data.columns.isin(["wage","lwage","sex","shs","hsg","mw","so","we","ne","exp1"])]
table = Z_1.mean(axis=0)
table

wage     23.410410
lwage     2.970787
sex       0.444466
shs       0.023301
hsg       0.243883
mw        0.259612
so        0.296505
we        0.216117
ne        0.227767
exp1     13.760583
dtype: float64

In [8]:
table = pd.DataFrame(data=table, columns={"Sample mean":"0"} )
table.index
index1 = list(table.index)
index2 = ["Log Wage","Sex","Some High School","High School Graduate","Midwest",
          "South","West","Northeast","Experience"]

In [9]:
table = table.rename(index=dict(zip(index1,index2)))
table

,Sample mean
Log Wage,23.410410
Sex,2.970787
Some High School,0.444466
High School Graduate,0.023301
Midwest,0.243883
South,0.259612
West,0.296505
Northeast,0.216117
Experience,0.227767
exp1,13.760583


In [10]:
print(table.to_latex())

\begin{tabular}{lr}
\toprule
{} &  Sample mean \\
\midrule
Log Wage             &    23.410410 \\
Sex                  &     2.970787 \\
Some High School     &     0.444466 \\
High School Graduate &     0.023301 \\
Midwest              &     0.243883 \\
South                &     0.259612 \\
West                 &     0.296505 \\
Northeast            &     0.216117 \\
Experience           &     0.227767 \\
exp1                 &    13.760583 \\
\bottomrule
\end{tabular}



In [11]:
# Import packages for OLS regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

#### 2.2 Basic model

In [12]:
flex = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'
flex_results_0 = smf.ols(flex , data=Z)
flex_results = smf.ols(flex , data=Z).fit()
print(flex_results.summary()) # estimated coefficients
print( "Number of regressors in the basic model:",len(flex_results.params), '\n')

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.180
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     6.212
Date:                Fri, 17 Sep 2021   Prob (F-statistic):           9.07e-33
Time:                        12:34:22   Log-Likelihood:                -872.87
No. Observations:                1376   AIC:                             1842.
Df Residuals:                    1328   BIC:                             2093.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0216      0.062     32.368      0.0

Note that the basic model consists of $51$ regressors

#### 2.3 Flexible model

In [13]:
flex = 'lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'
flex_results_0 = smf.ols(flex , data=Z)
flex_results = smf.ols(flex , data=Z).fit()
print(flex_results.summary()) # estimated coefficients
print( "Number of regressors in the basic model:",len(flex_results.params), '\n') # number of regressors in the Flexible Model

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     1.840
Date:                Fri, 17 Sep 2021   Prob (F-statistic):           2.24e-15
Time:                        12:34:27   Log-Likelihood:                -522.96
No. Observations:                1376   AIC:                             2034.
Df Residuals:                     882   BIC:                             4616.
Df Model:                         493                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 3.63

Note that the basic model consists of $979$ regressors

#### 2.4 Lasso Model

In [14]:
from sklearn.linear_model import LassoCV
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

In [15]:
# Get exogenous variables from flexible model
X = flex_results_0.exog
X

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [16]:
# Set endogenous variable
lwage = Z["lwage"]
lwage.shape

(1376,)

With the default value (0.1) for $\lambda$ the model do not converge, therefore we tried different values for $\lambda$ (0.1 0.2 0.3 0.4 and 0.5). The values 0.4 and 0.5 makes the model converge, however we decided to keep and work with the smaller value.


In [44]:
#lambda
alpha=0.4

In [45]:
# Set penalty value = 0.4
#reg = linear_model.Lasso(alpha=0.4/np.log(len(lwage)))

reg = linear_model.Lasso(alpha = alpha)

reg.fit(X, lwage)
lwage_lasso_fitted = reg.fit(X, lwage).predict( X )

reg.coef_
print('Lasso Regression: R^2 score', reg.score(X, lwage) ,reg.fit(X, lwage))

Lasso Regression: R^2 score 0.054428042189907044 Lasso(alpha=0.4)


In [19]:
lwage_lasso_fitted

array([2.71227413, 2.88906161, 2.94330572, ..., 2.68650772, 2.66540924,
       2.6599747 ])

In [20]:
# Basic Model
basic = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'
basic_results = smf.ols(basic , data=Z).fit()

# Flexible model 
flex = 'lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'
flex_results = smf.ols(flex , data=Z).fit()

In [21]:
# Assess the predictive performance
R2_1 = basic_results.rsquared
print("R-squared for the basic model: ", R2_1, "\n")
R2_adj1 = basic_results.rsquared_adj
print("adjusted R-squared for the basic model: ", R2_adj1, "\n")


R2_2 = flex_results.rsquared
print("R-squared for the basic model: ", R2_2, "\n")
R2_adj2 = flex_results.rsquared_adj
print("adjusted R-squared for the basic model: ", R2_adj2, "\n")

R2_L = reg.score(flex_results_0.exog, lwage)
print("R-squared for LASSO: ", R2_L, "\n")
R2_adjL = 1 - (1-R2_L)*(len(lwage)-1)/(len(lwage)-X.shape[1]-1)
print("adjusted R-squared for LASSO: ", R2_adjL, "\n")

R-squared for the basic model:  0.18023814876721034 

adjusted R-squared for the basic model:  0.15122549288773657 

R-squared for the basic model:  0.5070440013634905 

adjusted R-squared for the basic model:  0.23150283659274318 

R-squared for LASSO:  0.054428042189907044 

adjusted R-squared for LASSO:  -2.2832359646183784 



In [22]:
# calculating the MSE
MSE1 =  np.mean(basic_results.resid**2)
print("MSE for the basic model: ", MSE1, "\n")
p1 = len(basic_results.params) # number of regressors
n = len(lwage)
MSE_adj1  = (n/(n-p1))*MSE1
print("adjusted MSE for the basic model: ", MSE_adj1, "\n")

MSE2 =  np.mean(flex_results.resid**2)
print("MSE for the flexible model: ", MSE2, "\n")
p2 = len(flex_results.params) # number of regressors
n = len(lwage)
MSE_adj2  = (n/(n-p2))*MSE2
print("adjusted MSE for the flexible model: ", MSE_adj2, "\n")


MSEL = mean_squared_error(lwage, lwage_lasso_fitted)
print("MSE for the LASSO model: ", MSEL, "\n")
pL = reg.coef_.shape[0] # number of regressors
n = len(lwage)
MSE_adjL  = (n/(n-pL))*MSEL
print("adjusted MSE for LASSO model: ", MSE_adjL, "\n")

MSE for the basic model:  0.20821908377460777 

adjusted MSE for the basic model:  0.21623355416895118 

MSE for the flexible model:  0.12521056721892154 

adjusted MSE for the flexible model:  0.4339791951970681 

MSE for the LASSO model:  0.24017478539907006 

adjusted MSE for LASSO model:  0.8324445962446358 



In [23]:
import array_to_latex as a2l

table = np.zeros((3, 5))
table[0,0:5] = [p1, R2_1, MSE1, R2_adj1, MSE_adj1]
table[1,0:5] = [p2, R2_2, MSE2, R2_adj2, MSE_adj2]
table[2,0:5] = [pL, R2_L, MSEL, R2_adjL, MSE_adjL]
table

array([[ 5.10000000e+01,  1.80238149e-01,  2.08219084e-01,
         1.51225493e-01,  2.16233554e-01],
       [ 9.79000000e+02,  5.07044001e-01,  1.25210567e-01,
         2.31502837e-01,  4.33979195e-01],
       [ 9.79000000e+02,  5.44280422e-02,  2.40174785e-01,
        -2.28323596e+00,  8.32444596e-01]])

In [24]:
table = pd.DataFrame(table, columns = ["p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$"], \
                      index = ["basic reg","flexible reg", "lasso flex"])
table

,p,$R^2_{sample}$,$MSE_{sample}$,$R^2_{adjusted}$,$MSE_{adjusted}$
basic reg,51.0,0.180238,0.208219,0.151225,0.216234
flexible reg,979.0,0.507044,0.125211,0.231503,0.433979
lasso flex,979.0,0.054428,0.240175,-2.283236,0.832445


Considering all measures above, the flexible model performs better than the basic model. And the basic model performs better than the lasso model. 


#### 2.5 Data splitting

In [25]:
np.random.seed(0)
random = np.random.randint(0,n, size=math.floor(n))
Z["random"] = random
random

<ipython-input-25-5aa61109b4b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Z["random"] = random


array([ 684,  559, 1216, ..., 1294,  573, 1367])

In [26]:
Z_2 = Z.sort_values(by=['random'])
Z_2.head() #sort the random numbers in order

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,...,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2,random
rownames,,,,,,,,,,,,,,,,,,,,,
9262,9.134615,2.212071,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,5.0,0.25,0.125,0.0625,9360,22,5090,9,0
14134,24.038462,3.179655,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,11.0,1.21,1.331,1.4641,4250,14,7770,16,0
8689,12.980769,2.563469,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,17.0,2.89,4.913,8.3521,5620,17,5380,9,3
11486,12.980769,2.563469,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,7.0,0.49,0.343,0.2401,4760,16,5790,9,3
20866,34.615385,3.544298,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,5.0,0.25,0.125,0.0625,220,1,770,4,3


In [27]:
#dividing the data
train = Z_2[ : math.floor(n*4/5)]    
test =  Z_2[ math.floor(n*4/5) : ]   
print(train.shape)
print(test.shape)

(1100, 21)
(276, 21)


In [28]:
# Basic Model
basic = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'
basic_results = smf.ols(basic , data=Z).fit()

# Flexible model 
flex = 'lwage ~ (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'
flex_results = smf.ols(flex , data=Z).fit()

In [29]:
basic_results = smf.ols(basic , data=train).fit()
print(basic_results.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     5.265
Date:                Fri, 17 Sep 2021   Prob (F-statistic):           2.13e-25
Time:                        12:45:27   Log-Likelihood:                -711.67
No. Observations:                1100   AIC:                             1519.
Df Residuals:                    1052   BIC:                             1759.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0547      0.071     28.859      0.0

In [30]:
lwage_test = test["lwage"].values
#test = test.drop(columns=['wage', 'lwage', 'random'])
lwage_test

array([2.66882949, 2.45893116, 1.75270541, 2.48490665, 2.66882949,
       2.79399263, 2.7524162 , 2.7080502 , 2.48650793, 2.6192547 ,
       2.48650793, 2.87312995, 3.39476649, 2.82303731, 1.68354588,
       2.97592965, 3.00530172, 2.69080839, 3.0816747 , 1.5293952 ,
       2.82298017, 2.31215454, 2.15800386, 2.66882949, 2.39689577,
       1.68808912, 2.95651156, 2.3687249 , 2.26336438, 2.21207109,
       2.15800386, 2.73336801, 2.1092137 , 2.26336438, 2.49220595,
       3.17965511, 2.6762096 , 1.96611286, 3.55809155, 2.90521827,
       2.82298017, 2.64143051, 1.66552738, 2.44568594, 2.44568594,
       2.95651156, 3.43993821, 2.56346897, 2.66882949, 3.17965511,
       2.48650793, 2.95651156, 2.35867456, 1.75253876, 2.97631419,
       2.26336438, 2.35867456, 2.82298017, 1.5702172 , 3.45728685,
       2.26336438, 3.36197667, 3.02883222, 2.66882949, 3.76744178,
       2.66882949, 2.66882949, 3.17965511, 3.51612735, 2.35867456,
       2.73336801, 3.36197667, 3.45728685, 2.62003932, 2.26336

In [31]:
test = sm.add_constant(test)    
lwage_pred =  basic_results.predict(test) 
print(lwage_pred)

rownames
30171    2.849302
20062    2.635376
26099    2.518647
29638    2.849302
24679    2.917170
           ...   
5201     2.528900
9639     2.805803
15695    2.242189
3853     2.914903
20067    2.408205
Length: 276, dtype: float64


In [32]:
MSE_test1 = np.sum((lwage_test-lwage_pred)**2)/len(lwage_test)
R2_test1  = 1 - MSE_test1/np.var(lwage_test)

print("Test MSE for the basic model: ", MSE_test1, " ")
print("Test R2 for the basic model: ", R2_test1)

Test MSE for the basic model:  0.19987102511489846  
Test R2 for the basic model:  0.04156410192334081


In [33]:
# Flexible model
# estimating the parameters in the training sample
flex_results = smf.ols(flex , data=train).fit()

# calculating the out-of-sample MSE
lwage_flex_pred =  flex_results.predict(test) # predict out of sample
lwage_test = test["lwage"].values

MSE_test2 = np.sum((lwage_test-lwage_flex_pred)**2)/len(lwage_test)
R2_test2  = 1 - MSE_test2/np.var(lwage_test)

print("Test MSE for the flexible model: ", MSE_test2, " ")
print("Test R2 for the flexible model: ", R2_test2)

Test MSE for the flexible model:  21.548829286539082  
Test R2 for the flexible model:  -102.33249423157736


In [34]:
flex_results_0 = smf.ols(flex , data=train)
X_train = flex_results_0.exog
print(X_train.shape)

# Get endogenous variable 
lwage_train = train["lwage"]
print(lwage_train.shape)

(1100, 979)
(1100,)


In [35]:
# get exogenous variables from testing data used in flex model
flex_results_1 = smf.ols(flex , data=test)
X_test = flex_results_1.exog
print(X_test.shape)

# Get endogenous variable 
lwage_test = test["lwage"]
print(lwage_test.shape) # have 30% of the observations in test set

(276, 979)
(276,)


In [39]:
reg = linear_model.Lasso(alpha=0.5)
lwage_lasso_fitted = reg.fit(X_train, lwage_train).predict( X_test )

MSE_lasso = np.sum((lwage_test-lwage_lasso_fitted)**2)/len(lwage_test)
R2_lasso  = 1 - MSE_lasso/np.var(lwage_test)

print("Test MSE for the flexible model: ", MSE_lasso, " ")
print("Test R2 for the flexible model: ", R2_lasso)

#In this part of the paper we also have some problems with the convergence of the model, therefore we add the penalty and change
#the value of /lambda to 0.5

Test MSE for the flexible model:  0.20333225686544204  
Test R2 for the flexible model:  0.02496655478324572


In [40]:
table2 = np.zeros((3, 2))
table2[0,0] = MSE_test1
table2[1,0] = MSE_test2
table2[2,0] = MSE_lasso
table2[0,1] = R2_test1
table2[1,1] = R2_test2
table2[2,1] = R2_lasso

table2 = pd.DataFrame(table2, columns = ["$MSE_{test}$", "$R^2_{test}$"], \
                      index = ["basic reg","flexible reg","lasso regression"])
table2

,$MSE_{test}$,$R^2_{test}$
basic reg,0.199871,0.041564
flexible reg,21.548829,-102.332494
lasso regression,0.203332,0.024967


The best model is the basic one, follow by the lasso model. The flexible have really extrange outcome of the values, this can be explained because we have too many covariates for a few observation

### Partialling Out with Lasso

#### 3.1 Partialling out using lasso Matriz W=(exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2)

In [46]:
flex_y = 'lwage ~  exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'

flex_d = 'sex ~ exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'

In [54]:

from sklearn import linear_model
lasso_model = linear_model.Lasso( alpha = 0.4 )

# regression y with W 
flex_y_covariables = smf.ols(formula = flex_y, data = Z)
Y_lasso_fitted = lasso_model.fit( flex_y_covariables.exog, Z[[ 'lwage' ]] ).predict( flex_y_covariables.exog )
t_Y = Z[[ 'lwage' ]] - Y_lasso_fitted.reshape( Y_lasso_fitted.size, 1) #keep the residuals

#regression d with W
flex_d_covariables = smf.ols( flex_d, data=Z)
D_lasso_fitted = lasso_model.fit( flex_d_covariables.exog, Z[[ 'sex' ]] ).predict( flex_d_covariables.exog )
t_D = Z[[ 'sex' ]] - D_lasso_fitted.reshape( D_lasso_fitted.size, 1) #keep the residuals


In [55]:
data_res = pd.DataFrame( np.hstack(( t_Y , t_D )) , columns = [ 't_Y', 't_D' ] )

# regression of the residuals 
partial_lasso_fit = smf.ols( formula = 't_Y ~ t_D' , data = data_res ).fit()
partial_lasso_est = partial_lasso_fit.summary2().tables[1]['Coef.']['t_D']

print( f"Coefficient for D via partialling-out using lasso {partial_lasso_est}" )


Coefficient for D via partialling-out using lasso -0.12187773188853765


The use of the lasso for partialling-out the basic regressions shows us that the gender gap is of  $12.18$%

#### 3.2 Partialling out using lasso Matrix W = (exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2

In [57]:
flex_y = 'lwage ~(exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'

flex_d = 'sex ~(exp1+exp2+exp3+exp4+shs+hsg+scl+clg+occ2+ind2+mw+so+we)**2'

In [59]:
from sklearn import linear_model
lasso_model = linear_model.Lasso( alpha = 0.4 )

# regression y with W 
flex_y_covariables = smf.ols(formula = flex_y, data = Z)
Y_lasso_fitted = lasso_model.fit( flex_y_covariables.exog, Z[[ 'lwage' ]] ).predict( flex_y_covariables.exog )
t_Y = Z[[ 'lwage' ]] - Y_lasso_fitted.reshape( Y_lasso_fitted.size, 1) #keep the residuals

#regression d with W
flex_d_covariables = smf.ols( flex_d, data=Z)
D_lasso_fitted = lasso_model.fit( flex_d_covariables.exog, Z[[ 'sex' ]] ).predict( flex_d_covariables.exog )
t_D = Z[[ 'sex' ]] - D_lasso_fitted.reshape( D_lasso_fitted.size, 1) #keep the residuals

In [60]:
data_res = pd.DataFrame( np.hstack(( t_Y , t_D )) , columns = [ 't_Y', 't_D' ] )

# regression of the residuals 
partial_lasso_fit = smf.ols( formula = 't_Y ~ t_D' , data = data_res ).fit()
partial_lasso_est = partial_lasso_fit.summary2().tables[1]['Coef.']['t_D']

print( f"Coefficient for D via partialling-out using lasso {partial_lasso_est}" )


Coefficient for D via partialling-out using lasso -0.11070244242931254


The use of the lasso for partialling-out the basic regressions shows us that the gender gap is of  $11.07$%